In [1]:
import lightgbm as lgb
from catboost import CatBoostRegressor
import os, glob
import json
import folium
import pandas as pd
import numpy as np
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from sklearn.metrics import mean_absolute_error
from math import sqrt
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.preprocessing import LabelEncoder,MinMaxScaler,OneHotEncoder,StandardScaler



import warnings
# Filter out specific ValueWarnings from statsmodels
warnings.filterwarnings("ignore")

In [2]:
# !ls /kaggle/input/

In [3]:
ndays=1
exps_dir = "../../exps"
if os.path.exists(exps_dir) == False: # tạo thư mục (nếu chưa có)
  os.makedirs(exps_dir, exist_ok=True)

save_dir = f"{exps_dir}/exp1"
os.makedirs(save_dir, exist_ok=True)

In [4]:
df_train_features=pd.read_csv(f'{save_dir}/train_lag{ndays}.csv')
df_train_features

,row_id,county,is_business,product_type,is_consumption,prediction_unit_id,date,lowest_price_per_mwh,highest_price_per_mwh,origin_date,...,day,weekday,month,year,segment,sin(dayofyear),cos(dayofyear),sin(hour),cos(hour),target
0,366048,0,0,1,0,0,2022-01-01,80.0,94.0,2021-12-30,...,1,6,1,2022,0_0_1_0,0.017166,0.999853,0.000000,1.000000,0.000
1,366049,0,0,1,1,0,2022-01-01,80.0,94.0,2021-12-30,...,1,6,1,2022,0_0_1_1,0.017166,0.999853,0.000000,1.000000,442.226
2,366050,0,0,2,0,1,2022-01-01,80.0,94.0,2021-12-30,...,1,6,1,2022,0_0_2_0,0.017166,0.999853,0.000000,1.000000,0.000
3,366051,0,0,2,1,1,2022-01-01,80.0,94.0,2021-12-30,...,1,6,1,2022,0_0_2_1,0.017166,0.999853,0.000000,1.000000,44.899
4,366052,0,0,3,0,2,2022-01-01,80.0,94.0,2021-12-30,...,1,6,1,2022,0_0_3_0,0.017166,0.999853,0.000000,1.000000,0.015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1651897,2018347,15,1,0,1,64,2023-05-31,29.0,34.0,2023-05-29,...,31,3,5,2023,15_1_0_1,0.522133,-0.852864,-0.258819,0.965926,197.233
1651898,2018348,15,1,1,0,59,2023-05-31,29.0,34.0,2023-05-29,...,31,3,5,2023,15_1_1_0,0.522133,-0.852864,-0.258819,0.965926,0.000
1651899,2018349,15,1,1,1,59,2023-05-31,29.0,34.0,2023-05-29,...,31,3,5,2023,15_1_1_1,0.522133,-0.852864,-0.258819,0.965926,28.404
1651900,2018350,15,1,3,0,60,2023-05-31,29.0,34.0,2023-05-29,...,31,3,5,2023,15_1_3_0,0.522133,-0.852864,-0.258819,0.965926,0.000


In [5]:
df_train_features.drop(columns=['row_id'])

,county,is_business,product_type,is_consumption,prediction_unit_id,date,lowest_price_per_mwh,highest_price_per_mwh,origin_date,data_block_id,...,day,weekday,month,year,segment,sin(dayofyear),cos(dayofyear),sin(hour),cos(hour),target
0,0,0,1,0,0,2022-01-01,80.0,94.0,2021-12-30,122.0,...,1,6,1,2022,0_0_1_0,0.017166,0.999853,0.000000,1.000000,0.000
1,0,0,1,1,0,2022-01-01,80.0,94.0,2021-12-30,122.0,...,1,6,1,2022,0_0_1_1,0.017166,0.999853,0.000000,1.000000,442.226
2,0,0,2,0,1,2022-01-01,80.0,94.0,2021-12-30,122.0,...,1,6,1,2022,0_0_2_0,0.017166,0.999853,0.000000,1.000000,0.000
3,0,0,2,1,1,2022-01-01,80.0,94.0,2021-12-30,122.0,...,1,6,1,2022,0_0_2_1,0.017166,0.999853,0.000000,1.000000,44.899
4,0,0,3,0,2,2022-01-01,80.0,94.0,2021-12-30,122.0,...,1,6,1,2022,0_0_3_0,0.017166,0.999853,0.000000,1.000000,0.015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1651897,15,1,0,1,64,2023-05-31,29.0,34.0,2023-05-29,637.0,...,31,3,5,2023,15_1_0_1,0.522133,-0.852864,-0.258819,0.965926,197.233
1651898,15,1,1,0,59,2023-05-31,29.0,34.0,2023-05-29,637.0,...,31,3,5,2023,15_1_1_0,0.522133,-0.852864,-0.258819,0.965926,0.000
1651899,15,1,1,1,59,2023-05-31,29.0,34.0,2023-05-29,637.0,...,31,3,5,2023,15_1_1_1,0.522133,-0.852864,-0.258819,0.965926,28.404
1651900,15,1,3,0,60,2023-05-31,29.0,34.0,2023-05-29,637.0,...,31,3,5,2023,15_1_3_0,0.522133,-0.852864,-0.258819,0.965926,0.000


In [6]:
df_train_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1651902 entries, 0 to 1651901
Columns: 107 entries, row_id to target
dtypes: float64(93), int64(10), object(4)
memory usage: 1.3+ GB


In [7]:
numeric_columns=(list(df_train_features.select_dtypes('number')))[0:len(list(df_train_features.select_dtypes('number')))-10]
numeric_columns.append('target')
numeric_columns.remove('row_id')
numeric_columns.remove('data_block_id')
numeric_columns.remove('data_block_id_right')
numeric_columns.remove('hours_ahead')
numeric_columns.remove('hours_ahead_fl_7d')
numeric_columns.remove('county')
numeric_columns.remove('product_type')
numeric_columns.remove('is_consumption')
numeric_columns.remove('prediction_unit_id')
numeric_columns.remove('is_business')


numeric_columns

['lowest_price_per_mwh',
 'highest_price_per_mwh',
 'eic_count',
 'installed_capacity',
 'euros_per_mwh',
 'temperature',
 'dewpoint',
 'cloudcover_high',
 'cloudcover_low',
 'cloudcover_mid',
 'cloudcover_total',
 '10_metre_u_wind_component',
 '10_metre_v_wind_component',
 'direct_solar_radiation',
 'surface_solar_radiation_downwards',
 'snowfall',
 'total_precipitation',
 'hours_ahead_fl',
 'temperature_fl',
 'dewpoint_fl',
 'cloudcover_high_fl',
 'cloudcover_low_fl',
 'cloudcover_mid_fl',
 'cloudcover_total_fl',
 '10_metre_u_wind_component_fl',
 '10_metre_v_wind_component_fl',
 'direct_solar_radiation_fl',
 'surface_solar_radiation_downwards_fl',
 'snowfall_fl',
 'total_precipitation_fl',
 'hours_ahead_fd_7d',
 'temperature_fd_7d',
 'dewpoint_fd_7d',
 'cloudcover_high_fd_7d',
 'cloudcover_low_fd_7d',
 'cloudcover_mid_fd_7d',
 'cloudcover_total_fd_7d',
 '10_metre_u_wind_component_fd_7d',
 '10_metre_v_wind_component_fd_7d',
 'direct_solar_radiation_fd_7d',
 'surface_solar_radiation_do

In [8]:
category_columns=list(df_train_features.select_dtypes('category'))
category_columns.append('data_block_id')
category_columns.append('data_block_id_right')
category_columns.append('hours_ahead')
category_columns.append('hours_ahead_fl_7d')
category_columns.append('county')
category_columns.append('row_id')
category_columns.append('product_type')
category_columns.append('is_consumption')
category_columns.append('prediction_unit_id')
category_columns.append('segment')
category_columns.append('is_business')

In [9]:
list(df_train_features.select_dtypes('object'))

['date', 'origin_date', 'origin_date_right', 'segment']

In [10]:
import pandas as pd

def aggregate_hourly_to_daily(df, numeric_columns):
    df['date'] = pd.to_datetime(df['date'])
    df['date'] = df['date'].dt.date
    df['origin_date'] = pd.to_datetime(df['origin_date'])
    df['origin_date'] = df['origin_date'].dt.date
    df['origin_date_right'] = pd.to_datetime(df['origin_date_right'])
    df['origin_date_right'] = df['origin_date_right'].dt.date

    grouped = df.groupby([
        # 'customer_id', 
        'county',
        'is_business', 
        'product_type', 
        'is_consumption', 
        'prediction_unit_id', 
        'date', 
        'data_block_id', 
        'data_block_id_right', 
        # 'hours_ahead', 
        # 'hours_ahead_fl_7d',
        'origin_date',
        'origin_date_right',
        'segment',
        "sin(dayofyear)",
        'cos(dayofyear)',
    ])

    agg_dict = {
        'target': 'sum',
        'lowest_price_per_mwh': 'min',
        'highest_price_per_mwh': 'max'
    }
    
    for col in numeric_columns[2:-1]:
        agg_dict[col] = 'mean'

    result = grouped.agg(agg_dict).reset_index()

    result = result.sort_values(by=['date','county','is_business', 'product_type'])
    
    return result




In [11]:
df = aggregate_hourly_to_daily(df_train_features, numeric_columns)
df

,county,is_business,product_type,is_consumption,prediction_unit_id,date,data_block_id,data_block_id_right,origin_date,origin_date_right,...,surface_pressure_hl_7d,cloudcover_total_hl_7d,cloudcover_low_hl_7d,cloudcover_mid_hl_7d,cloudcover_high_hl_7d,windspeed_10m_hl_7d,winddirection_10m_hl_7d,shortwave_radiation_hl_7d,direct_solar_radiation_hl_7d,diffuse_radiation_hl_7d
0,0,0,1,0,0,2022-01-01,122.0,122.0,2021-12-30,2021-12-30,...,993.287497,71.611111,63.798611,32.465278,29.888889,4.349151,252.034723,11.583333,3.208333,8.375000
516,0,0,1,1,0,2022-01-01,122.0,122.0,2021-12-30,2021-12-30,...,993.287497,71.611111,63.798611,32.465278,29.888889,4.349151,252.034723,11.583333,3.208333,8.375000
1032,0,0,2,0,1,2022-01-01,122.0,122.0,2021-12-30,2021-12-30,...,993.287497,71.611111,63.798611,32.465278,29.888889,4.349151,252.034723,11.583333,3.208333,8.375000
1548,0,0,2,1,1,2022-01-01,122.0,122.0,2021-12-30,2021-12-30,...,993.287497,71.611111,63.798611,32.465278,29.888889,4.349151,252.034723,11.583333,3.208333,8.375000
2064,0,0,3,0,2,2022-01-01,122.0,122.0,2021-12-30,2021-12-30,...,993.287497,71.611111,63.798611,32.465278,29.888889,4.349151,252.034723,11.583333,3.208333,8.375000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66781,15,1,0,1,64,2023-05-31,637.0,637.0,2023-05-29,2023-05-29,...,1004.549091,58.109091,4.636364,74.890909,30.036364,2.254040,247.036364,139.963636,66.909091,73.054545
67297,15,1,1,0,59,2023-05-31,637.0,637.0,2023-05-29,2023-05-29,...,1004.549091,58.109091,4.636364,74.890909,30.036364,2.254040,247.036364,139.963636,66.909091,73.054545
67813,15,1,1,1,59,2023-05-31,637.0,637.0,2023-05-29,2023-05-29,...,1004.549091,58.109091,4.636364,74.890909,30.036364,2.254040,247.036364,139.963636,66.909091,73.054545
68329,15,1,3,0,60,2023-05-31,637.0,637.0,2023-05-29,2023-05-29,...,1004.549091,58.109091,4.636364,74.890909,30.036364,2.254040,247.036364,139.963636,66.909091,73.054545


In [12]:
# for 'segment' in category_columns:
label_encoder = LabelEncoder()
df['segment'] = label_encoder.fit_transform(df['segment'])
#     df_train_features['segment'] = label_encoder  

In [13]:
scaler = StandardScaler()
df[numeric_columns] = scaler.fit_transform(df[numeric_columns])
df

,county,is_business,product_type,is_consumption,prediction_unit_id,date,data_block_id,data_block_id_right,origin_date,origin_date_right,...,surface_pressure_hl_7d,cloudcover_total_hl_7d,cloudcover_low_hl_7d,cloudcover_mid_hl_7d,cloudcover_high_hl_7d,windspeed_10m_hl_7d,winddirection_10m_hl_7d,shortwave_radiation_hl_7d,direct_solar_radiation_hl_7d,diffuse_radiation_hl_7d
0,0,0,1,0,0,2022-01-01,122.0,122.0,2021-12-30,2021-12-30,...,-1.098545,0.364756,0.540221,-0.102747,-0.219846,0.145623,0.765906,-1.098287,-0.907099,-1.263447
516,0,0,1,1,0,2022-01-01,122.0,122.0,2021-12-30,2021-12-30,...,-1.098545,0.364756,0.540221,-0.102747,-0.219846,0.145623,0.765906,-1.098287,-0.907099,-1.263447
1032,0,0,2,0,1,2022-01-01,122.0,122.0,2021-12-30,2021-12-30,...,-1.098545,0.364756,0.540221,-0.102747,-0.219846,0.145623,0.765906,-1.098287,-0.907099,-1.263447
1548,0,0,2,1,1,2022-01-01,122.0,122.0,2021-12-30,2021-12-30,...,-1.098545,0.364756,0.540221,-0.102747,-0.219846,0.145623,0.765906,-1.098287,-0.907099,-1.263447
2064,0,0,3,0,2,2022-01-01,122.0,122.0,2021-12-30,2021-12-30,...,-1.098545,0.364756,0.540221,-0.102747,-0.219846,0.145623,0.765906,-1.098287,-0.907099,-1.263447
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66781,15,1,0,1,64,2023-05-31,637.0,637.0,2023-05-29,2023-05-29,...,-0.192621,-0.064801,-1.187006,1.462371,-0.214599,-1.127396,0.696150,0.231730,-0.055061,0.876067
67297,15,1,1,0,59,2023-05-31,637.0,637.0,2023-05-29,2023-05-29,...,-0.192621,-0.064801,-1.187006,1.462371,-0.214599,-1.127396,0.696150,0.231730,-0.055061,0.876067
67813,15,1,1,1,59,2023-05-31,637.0,637.0,2023-05-29,2023-05-29,...,-0.192621,-0.064801,-1.187006,1.462371,-0.214599,-1.127396,0.696150,0.231730,-0.055061,0.876067
68329,15,1,3,0,60,2023-05-31,637.0,637.0,2023-05-29,2023-05-29,...,-0.192621,-0.064801,-1.187006,1.462371,-0.214599,-1.127396,0.696150,0.231730,-0.055061,0.876067


In [14]:
# df_train_features.drop('row_id',axis=1)

In [15]:
df.to_csv(f'{save_dir}/train_lag{ndays}_days_pre_processing.csv')

In [16]:
df.segment

0         0
516       1
1032      2
1548      3
2064      4
         ..
66781    63
67297    64
67813    65
68329    66
68845    67
Name: segment, Length: 68846, dtype: int32